In [2]:
from mpi4py import MPI
import numpy as np
import time

N = 1000  # Ukuran matriks

# Inisialisasi MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
numproc = comm.Get_size()

# Inisialisasi matriks
if rank == 0:
    np.random.seed(int(time.time()))
    A = np.random.randint(0, 10, (N, N)).astype(np.float64)
    B = np.random.randint(0, 10, (N, N)).astype(np.float64)
    C_seq = np.zeros((N, N), dtype=np.float64)
    C_par = np.zeros((N, N), dtype=np.float64)
else:
    A = np.empty((N, N), dtype=np.float64)
    B = np.empty((N, N), dtype=np.float64)
    C_par = np.zeros((N, N), dtype=np.float64)

# Broadcast matriks A dan B ke semua proses
comm.Bcast(A, root=0)
comm.Bcast(B, root=0)

# Fungsi perkalian matriks secara sequential
def sequential_multiply(A, B):
    return np.dot(A, B)

# Fungsi perkalian matriks secara parallel
def parallel_multiply(A, B, rank, numproc):
    rows_per_process = N // numproc
    start = rank * rows_per_process
    end = N if rank == numproc - 1 else start + rows_per_process

    C_partial = np.dot(A[start:end, :], B)
    return C_partial

# Rank 0 menjalankan sequential
if rank == 0:
    start_time = time.time()
    C_seq = sequential_multiply(A, B)
    seq_time = time.time() - start_time
    print(f"Sequential execution time: {seq_time:.6f} seconds")

# Parallel Execution
start_time = time.time()
C_partial = parallel_multiply(A, B, rank, numproc)

# Mengumpulkan hasil dari semua proses
C_par = np.empty((N, N), dtype=np.float64) if rank == 0 else None
comm.Gather(C_partial, C_par, root=0)

# Rank 0 mencetak waktu eksekusi parallel
if rank == 0:
    par_time = time.time() - start_time
    print(f"Parallel execution time using {numproc} processes: {par_time:.6f} seconds")

Sequential execution time: 0.028005 seconds
Parallel execution time using 1 processes: 0.033003 seconds
